In [ ]:
# 🛠 Install dependencies
pip install -q gradio beautifulsoup4 requests

import requests
import gradio as gr
from bs4 import BeautifulSoup

# 🔑 Add your Gemini API key here
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"

# 🔍 Fetch PubMed abstracts
def fetch_pubmed_abstracts(query, max_results=3):
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

    # Search for relevant articles
    search_params = {
        "db": "pubmed",
        "term": query,
        "retmode": "json",
        "retmax": max_results
    }
    search_res = requests.get(search_url, params=search_params).json()
    ids = search_res.get("esearchresult", {}).get("idlist", [])

    if not ids:
        return "No relevant articles found."

    # Fetch abstracts
    fetch_params = {
        "db": "pubmed",
        "id": ",".join(ids),
        "retmode": "xml"
    }
    fetch_res = requests.get(fetch_url, params=fetch_params)
    soup = BeautifulSoup(fetch_res.text, "xml")

    abstracts = []
    for article in soup.find_all("PubmedArticle"):
        title = article.ArticleTitle.text if article.ArticleTitle else ""
        abstract = article.Abstract.AbstractText.text if article.Abstract and article.Abstract.AbstractText else "No abstract available."
        abstracts.append(f"Title: {title}\nAbstract: {abstract}")

    return "\n\n".join(abstracts)

# 🧠 Ask Gemini
def ask_gemini(question):
    context = fetch_pubmed_abstracts(question)
    if context.startswith("No relevant"):
        return context

    prompt = f"""You are a helpful medical assistant. Based on the following PubMed abstracts, answer the user's question.

Context:
{context}

Question:
{question}

Answer:"""

    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [{
            "parts": [{"text": prompt}]
        }]
    }

    response = requests.post(GEMINI_URL, headers=headers, json=payload)
    if response.status_code == 200:
        try:
            return response.json()['candidates'][0]['content']['parts'][0]['text']
        except:
            return "⚠️ Error parsing Gemini's response."
    else:
        return f"❌ API Error {response.status_code}: {response.text}"

# 🎛️ Gradio UI
gr.Interface(
    fn=ask_gemini,
    inputs=gr.Textbox(label="Ask a Medical Question", placeholder="e.g., What are new treatments for depression?"),
    outputs=gr.Textbox(label="Answer", lines=12),
    title="🧬 Medical Chatbot using PubMed + Gemini",
    description="Enter a medical question. PubMed abstracts will be searched and used by Gemini to generate an answer."
).launch()
